## Simple Convolutional Neural Network
In this part, we learn to:
- train a simple CNN on MNIST dataset.

### 1. Loading Essential Packages

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


### 2. Seting the hyper-parameters Value

In [0]:
batch_size = 128    # Here we define the batch size value
num_classes = 10    # Assign the number of class exists in MNIST dataset
epochs = 12       # Total Number of iteratin on mnist dataset 
img_rows, img_cols = 28, 28        # input image dimensions

### 3. Loading MNIST Dataset
Uncomment bellow line and use mnist.load_data() function to load mnist dataset

In [3]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [5]:
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
!unzip hymenoptera_data.zip
!mv hymenoptera_data data

--2018-08-02 11:31:16--  https://download.pytorch.org/tutorial/hymenoptera_data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.35.115.103, 13.35.115.56, 13.35.115.18, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.35.115.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47286322 (45M) [application/zip]
Saving to: ‘hymenoptera_data.zip’

hymenoptera_data.zi 100%[===================>]  45.10M  26.8MB/s    in 1.7s    

2018-08-02 11:31:18 (26.8 MB/s) - ‘hymenoptera_data.zip’ saved [47286322/47286322]

Archive:  hymenoptera_data.zip
   creating: hymenoptera_data/
   creating: hymenoptera_data/train/
   creating: hymenoptera_data/train/ants/
  inflating: hymenoptera_data/train/ants/0013035.jpg  
  inflating: hymenoptera_data/train/ants/1030023514_aad5c608f9.jpg  
  inflating: hymenoptera_data/train/ants/1095476100_3906d8afde.jpg  
  inflating: hymenoptera_data/train/ants/1099452230_d1949d3250.jpg  
  inflating: hymenoptera_data/

  inflating: hymenoptera_data/val/bees/1181173278_23c36fac71.jpg  
  inflating: hymenoptera_data/val/bees/1297972485_33266a18d9.jpg  
  inflating: hymenoptera_data/val/bees/1328423762_f7a88a8451.jpg  
  inflating: hymenoptera_data/val/bees/1355974687_1341c1face.jpg  
  inflating: hymenoptera_data/val/bees/144098310_a4176fd54d.jpg  
  inflating: hymenoptera_data/val/bees/1486120850_490388f84b.jpg  
  inflating: hymenoptera_data/val/bees/149973093_da3c446268.jpg  
  inflating: hymenoptera_data/val/bees/151594775_ee7dc17b60.jpg  
  inflating: hymenoptera_data/val/bees/151603988_2c6f7d14c7.jpg  
  inflating: hymenoptera_data/val/bees/1519368889_4270261ee3.jpg  
  inflating: hymenoptera_data/val/bees/152789693_220b003452.jpg  
  inflating: hymenoptera_data/val/bees/177677657_a38c97e572.jpg  
  inflating: hymenoptera_data/val/bees/1799729694_0c40101071.jpg  
  inflating: hymenoptera_data/val/bees/181171681_c5a1a82ded.jpg  
  inflating: hymenoptera_data/val/bees/187130242_4593a4c610.jpg  
  i

### 4. This is an advance topic and not essential to be adept at this, but TAs know. Ask them!

In [0]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

### 5. Here we are going to understand:
1. How can we change variables type?
2. How can we normalized the numbers in a range between 0 and 1? This is a simple solution, But there are other ways. google it:)
3. How can we get the exact dimension of each variable

In [6]:
x_train = x_train.astype('float32')      # This is the way we change the variable type
x_test = x_test.astype('float32')
x_train /= 255                           # Here we normalize the data between 0 and 1
x_test /= 255                            # Here we normalize the data between 0 and 1
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


### 6. Using this way, you can find which classes each data point belongs to!

In [7]:
print(y_train[0])    # This refers to that the sample #1 is related to Class 5

5


### 7.Convert target representation from a simple scalar to one-hot representation:
One-Hot encoding. A one hot encoding is a representation of categorical variables as binary vectors. Each integer value is represented as a binary vector that is all zero values except the index of the integer, which is marked with a 1.

In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [9]:
print(y_train[0])    # above command assign 1 to 5th element of a vector and others have value 0.

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


### ->So far, all above operations are easy to understand. If you want to know more on each function operation, just use "shift + tab" on each function name. <-

### 8. Now, this is your turn. we are going to define a sequential model describer in following:
- Conv2D: filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = (1, 28, 28)
- Conv2D: filters = 64, kernel_size = (3,3), activation = 'relu'
- MaxPooling2D: pool_size=(2, 2)
- Dropout: rate = 0.25
- Flatten
- Dense: units = 128, activation='relu'
- Dropout: rate = 0.5
- Dense: units = 10, activation='softmax'

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3,3),input_shape = (28,28,1), strides=(1, 1), padding='valid', data_format=None, activation='relu', use_bias=True))
model.add(Conv2D(64, kernel_size = (3,3), strides=(1, 1), padding='valid', data_format=None, activation='relu', use_bias=True))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu', use_bias=True))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax', use_bias=True))

### 9. Same as before, we need to compile our above model with an optimizer and a caregorical loss function. You already know it.

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

### 10. Here, we try to fit our model on MNIST Dataset

In [14]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 12s 202us/step - loss: 0.2676 - acc: 0.9172 - val_loss: 0.0596 - val_acc: 0.9813
Epoch 2/12
60000/60000 [==============================] - 10s 165us/step - loss: 0.0889 - acc: 0.9736 - val_loss: 0.0450 - val_acc: 0.9850
Epoch 3/12
28416/60000 [=============>................] - ETA: 4s - loss: 0.0701 - acc: 0.979560000/60000 [==============================] - 10s 165us/step - loss: 0.0677 - acc: 0.9802 - val_loss: 0.0355 - val_acc: 0.9872
Epoch 4/12
60000/60000 [==============================] - 10s 165us/step - loss: 0.0549 - acc: 0.9841 - val_loss: 0.0326 - val_acc: 0.9883
Epoch 5/12
60000/60000 [==============================] - 10s 165us/step - loss: 0.0473 - acc: 0.9854 - val_loss: 0.0318 - val_acc: 0.9890
Epoch 6/12
  512/60000 [..............................] - ETA: 9s - loss: 0.0493 - acc: 0.9863 60000/60000 [==============================] - 10s 165us/step - loss: 0.0406 

In [15]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.02479468177455092
Test accuracy: 0.9919


## Pre-trained models and Data Augmentation:
In this section, we are going to use a pre-trained model called MobileNet. Using this network and a new dataset, we train a newly defined Convolutinal Neural Network. A comprehensive list of goals are presented bellow (Be cautious of What I list. Think about them)

1. Finetuning A pre-trained deep neural network
2. Training on a new dataset gave you before handson.
3. Data Augmenting using KERAS Utilities.

In [0]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

### 0. Hyper-parameters definition

In [0]:
img_width, img_height = 128, 128    # input image has size (3, 128, 128)
train_data_dir = "data/train"    # Location of training data
validation_data_dir = "data/val"    # Location of validation data
nb_train_samples = 244       # Total Number of Training samples
nb_validation_samples = 153       # Total Number of Validations samples
batch_size = 16
epochs = 50

### 1.Using commands introduced in hands on CNN, try to load MobileNet instead of VGG19. Just change the name ;)

In [55]:
from keras.applications.mobilenet import MobileNet
model = MobileNet(weights='imagenet',include_top= False,input_shape = (128,128,3))

17227776/17225924 [==============================] - 2s 0us/step


In [0]:
model.summary()

### 2. try to freeze just all of the layers in model included above. look at slides if you need

In [26]:
len(model.layers)

102

In [0]:
# Freeze the first five layers which you don't want to train. 
for i in range(0,len(model.layers)):   ######## You shold change this line ########
    model.layers[i] = False

### 3. Here we are going to attach the new classifier at the end of pretrained model. This is a new technique whcih we are going to explore more.
- Flatten Layer
- Desne Layer: units: 1024, activation = "relu"
- Dropout: rate = 0.5
- Desne Layer: units: 512, activation = "relu"
- Desne Layer: units: 2, activation = "softamax"

In [0]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation='relu', use_bias=True)(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', use_bias=True)(x)
x = Dense(2, activation='softmax', use_bias=True)(x)

In [0]:
# creating the final model 
model_final = Model(inputs = model.input, outputs = x)

In [0]:
model_final.summary()

In [0]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

### 4. Define a data augmentator as presented in CNN hands-on slides according to the following parameters:
- rescale = 1./255
- horizontal_flip = True
- fill_mode = "nearest"
- zoom_range = 0.6
- width_shift_range = 0.2
- height_shift_range=0.4
- rotation_range=25

In [0]:
# This an augmentator for test dataset
train_datagen = ImageDataGenerator(rescale = 1./255,
                    horizontal_flip = True,
                    fill_mode = "nearest",
                    zoom_range = 0.6,
                    width_shift_range = 0.2,
                    height_shift_range=0.4,
                    rotation_range=25)


valid_datagen = ImageDataGenerator(rescale = 1./255,
                    horizontal_flip = True,
                    fill_mode = "nearest",
                    zoom_range = 0.6,
                    width_shift_range = 0.2,
                    height_shift_range=0.4,
                    rotation_range=25)

### 5. in This section, What we are going to do is to create a dataloader for loading data along with augmentator.
1. Train Loader Setting

    - directory = train_data_dir
    - target_size = (img_height, img_width)
    - batch_size = batch_size
    - class_mode = "categorical"


----------------------------------------------------------
2. Test Loader Setting

    - directory = validation_data_dir,
    - target_size = (img_height, img_width)
    - class_mode = "categorical"

In [70]:
train_generator = train_datagen.flow_from_directory(
        directory = train_data_dir,
        target_size = (img_height, img_width),
        batch_size = batch_size,
        class_mode = "categorical")


Found 244 images belonging to 2 classes.


In [76]:
valid_generator = valid_datagen.flow_from_directory(
        directory = validation_data_dir,
        target_size = (img_height, img_width),
        batch_size = batch_size,
        class_mode = "categorical")


Found 153 images belonging to 2 classes.


In [0]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [79]:
# Train the model 
model_final.fit_generator(train_generator,
        samples_per_epoch = nb_train_samples,
        epochs = epochs,
        validation_data = valid_generator,
        nb_val_samples = nb_validation_samples,
        callbacks = [checkpoint, early])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, validation_data=<keras.pre..., callbacks=[<keras.ca..., steps_per_epoch=15, validation_steps=153)`
  


Epoch 1/50
15/15 [==============================] - 35s 2s/step - loss: 1.2485 - acc: 0.5217 - val_loss: 0.8491 - val_acc: 0.5830

Epoch 00001: val_acc improved from -inf to 0.58301, saving model to vgg16_1.h5
Epoch 2/50
15/15 [==============================] - 29s 2s/step - loss: 1.2166 - acc: 0.4908 - val_loss: 0.6597 - val_acc: 0.6714

Epoch 00002: val_acc improved from 0.58301 to 0.67136, saving model to vgg16_1.h5
Epoch 3/50
15/15 [==============================] - 29s 2s/step - loss: 0.9619 - acc: 0.5987 - val_loss: 0.5806 - val_acc: 0.7157

Epoch 00003: val_acc improved from 0.67136 to 0.71575, saving model to vgg16_1.h5
Epoch 4/50
15/15 [==============================] - 29s 2s/step - loss: 0.8450 - acc: 0.6583 - val_loss: 0.5650 - val_acc: 0.7315

Epoch 00004: val_acc improved from 0.71575 to 0.73154, saving model to vgg16_1.h5
Epoch 5/50
15/15 [==============================] - 29s 2s/step - loss: 0.8990 - acc: 0.5890 - val_loss: 0.6283 - val_acc: 0.7239

Epoch 00005: val_acc